In [2]:
from abc import ABC, abstractmethod


class Expression(ABC):  # an interface
    pass


class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend) -> Expression:
        return Money(self._amount + addend._amount, self._currency)
    

class Bank:


    def reduce(self, source: Expression, to: str):
        return Money.dollar(10)
    

def testSimpleAddition():
    five = Money.dollar(5)
    sum_: Expression = five + five
    bank = Bank()
    reduced: Money = bank.reduce(sum_, "USD")
    assert Money.dollar(10) == reduced


testSimpleAddition()

First, `Money.__add__()` needs to return a real `Expression`—a `Sum`, not just a `Money`.

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. **\\$5 + \\$5 = \\$10**
17. Return `Money` from \\$5 + \\$5

In [5]:
def testPlusReturnsSum():
    five = Money.dollar(5)
    result: Expression = five + five
    sum_: Sum = result  # in Java, cast (Sum) result
    assert five == sum_.augend
    assert five == sum_.addend


testPlusReturnsSum()

AttributeError: 'Money' object has no attribute 'augend'

To get it to compile, all we need is a `Sum` class:

In [6]:
class Sum(Expression):      # implement interface

    augend: Money | None = None
    addend: Money | None = None


    def __init__(self, augend, addend):
        self.augend = augend
        self.addend = addend


class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend) -> Expression:
        return Sum(self, addend)
    

testPlusReturnsSum()

Now `Bank.reduce()` is being passed a `Sum`. If the currencies in the `Sum` are all the
same, and the target currency is also the same, then the result should be a `Money`
whose amount is the sum of the amounts:

In [8]:
def testReduceSum():
    sum_ : Expression = Sum(Money.dollar(3), Money.dollar(4))
    bank: Bank = Bank()
    result = bank.reduce(sum_, "USD")
    assert Money.dollar(7) == result


testReduceSum()

AssertionError: 

In [9]:
class Bank:


    def reduce(self, source: Expression, to: str) -> Money:
        sum_ : Sum = source     # in Java, cast to (Sum) source
        return sum_.reduce(to)
    

class Sum(Expression):      # implement interface

    augend: Money | None = None
    addend: Money | None = None


    def __init__(self, augend, addend):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, to: str):
        amount : int = self.augend._amount + self.addend._amount
        return Money(amount, to)
    

testReduceSum()

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. **\\$5 + \\$5 = \\$10**
17. Return `Money` from \\$5 + \\$5
18. `Bank.reduce(Money)`

In [10]:
def testReduceMoney():
    bank : Bank = Bank()
    result : Money = bank.reduce(Money.dollar(1), "USD")
    assert Money.dollar(1) == result


testReduceMoney()

AttributeError: 'Money' object has no attribute 'reduce'

In [12]:
class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, to : str) -> Money:
        pass


class Bank:


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(to)
    

class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend) -> Expression:
        return Sum(self, addend)
    

    def reduce(self, to : str) -> Money:
        return self
    

testReduceMoney()

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. **\\$5 + \\$5 = \\$10**
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. Reduce `Money` with conversation
20. `Reduce(Money, String)`